In [ ]:
import csv
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### Data set loading

In [1]:
def load_csv(filename):
    file = open(filename, "r", encoding="utf-8")
    experiments = []
    headers = []
    reader = csv.reader(file)
    for i, lines in enumerate(reader):
        if i == 0:
            #headers = [e for e in lines[0].split(";")]
            h = lines[0].split(";")
            headers.append(h)
        else:
            tmp = lines[0].split(";")
            for j in range(0,len(tmp)):
                if (j != 0 and j != 1879):
                    tmp[j] = float(tmp[j])
                else:
                    continue
            #tmp = [float(tmp[j]) for j in range(0,len(tmp)) if (j!=0 and j!=1879)] #This removes two entries, dunno why. 
            experiments.append(tmp)
    file.close()
    return headers, experiments

In [2]:
headers, experiments =load_csv('/home/jovyan/covid-data/dalia_MeanFeaturesPerWell_BatchA.csv')
experiments_df = pd.DataFrame(experiments).iloc[::-1] #reversed dataframe

NameError: name 'csv' is not defined

In [ ]:
print(f'Number of experiments in MeanFeaturesPerWell_BatchA: {len(experiments)}') #Experiments in batch A
print(f'Number of columns in MeanFeaturesPerWell_BatchA: {len(experiments[0])}') #Columns

In [ ]:
headers[0][-5:]

### Principal component analysis: extract information

In [ ]:
pca_info_headers = [headers[i][1879:] for i in range(0,len(headers))]
print(pca_info_headers)
pca_info_data = [experiments[i][1879:] for i in range(0,len(experiments))]
print(pca_info_data[0:3])

### Maris' heatmaps replication

In [ ]:
def filter_plate(plate_prefix,data):
    plate = []
    for i in range(0, len(data)):
        if data[i][0].startswith(plate_prefix):
            plate.append(data[i])
    return plate

In [ ]:
#Filtering plate SS-001A-2
plate_SS_001A_2 = filter_plate('001A',experiments)
#print(len(plate_001A)) #make sure that no data is missing
#print(len(plate_001A[0]) )#make sure that no data is missing

#Preparing data
df = pd.DataFrame(plate_SS_001A_2)
sorted_df = df.sort_values(by=0)
sorted_df.head(5)

In [ ]:
def find_feature_index(feature):
    for i, name in enumerate(headers[0]):
        if feature == name:
            break;
    return i

def heatmap(feature_name,save=False):
    alphabet = [chr(i) for i in range(ord('A'),ord('P')+1)]
    feature_index = find_feature_index(feature_name)
    feature_data =  sorted_df[sorted_df.columns[feature_index]].to_numpy().reshape(16, 24)
    df_cm = pd.DataFrame(feature_data, index = alphabet, columns = np.arange(1,25))
    plt.figure(figsize = (7,4))
    sns.heatmap(df_cm, cmap = 'Purples', linewidth = 1, annot=False)
    plt.yticks(rotation=0)
    plt.title(feature_name)
    if save:
        plt.savefig("".join(["heatmap_",feature_name, ".png"]))
    plt.show()   

In [ ]:
heatmap("Intensity_MeanIntensity_illumMITO_cells",save=True) 

In [ ]:
heatmap("AreaShape_Area_nuclei")

In [ ]:
def scatter_feature(feature_name,feature_index, save=False):
    fig, ax = plt.subplots(1, 1, figsize=(9,5))
    g = sns.scatterplot(data=experiments_df[0:5463], x=0, y=feature_index, s=10, edgecolor='slateblue', linewidth=3, facecolor='slateblue')
    g = sns.scatterplot(data=experiments_df[5463:5651], x=0, y=feature_index, s=10, edgecolor='pink', linewidth=3, facecolor='pink')
    g = sns.scatterplot(data=experiments_df[5651:5809], x=0, y=feature_index, s=10, edgecolor='sandybrown', linewidth=3, facecolor='sandybrown')
    g = sns.scatterplot(data=experiments_df[5809:], x=0, y=feature_index, s=10, edgecolor='darkseagreen', linewidth=3, facecolor='darkseagreen')
    sns.set_style("ticks")
    g.set(xticklabels=[])
    g.tick_params(bottom=False)
    g.set_xlabel("ID", color="black", fontsize=15)
    g.set_ylabel(feature_name, color="black", fontsize=15)
    if save:
        plt.savefig("".join(["scatter_",feature_name, ".png"]))
    plt.show()

In [ ]:
scatter_feature(feature_name="Intensity_MeanIntensity_illumMITO_cells",feature_index=1882, save=True)